<a href="https://colab.research.google.com/github/AdityaaRavi/Butterfly-cnn/blob/main/Butteryfly_cnn_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the data

In [1]:
!mkdir /content/butterfly-data

In [14]:
# get the kaggle api token
!mkdir ~/.kaggle
file_id = "1zSRznA3TypBwqP75WcOuv6X3tbaYDiNJ"
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id
!wget -O ~/.kaggle/kaggle.json --no-check-certificate "$file_download_link"
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
--2022-11-20 19:30:15--  https://docs.google.com/uc?export=download&id=1zSRznA3TypBwqP75WcOuv6X3tbaYDiNJ
Resolving docs.google.com (docs.google.com)... 108.177.13.139, 108.177.13.138, 108.177.13.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.13.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q79rfjvsifmj9ueiid6heeegf81gptrc/1668972600000/09300047399178709700/*/1zSRznA3TypBwqP75WcOuv6X3tbaYDiNJ?e=download&uuid=79a72b4e-ea87-44b1-a535-d77d2c7b4af7 [following]
--2022-11-20 19:30:16--  https://doc-10-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q79rfjvsifmj9ueiid6heeegf81gptrc/1668972600000/09300047399178709700/*/1zSRznA3TypBwqP75WcOuv6X3tbaYDiNJ?e=download&uuid=79a72b4e-ea87-44b1-a535-d77d2c7b4af7
Resolving doc-10-bk-docs.googleusercontent.com (doc-10-

In [15]:
!kaggle datasets download -d gpiosenka/butterfly-images40-species -p /content/butterfly-data/ --unzip

 97% 440M/454M [00:03<00:00, 137MB/s]
100% 454M/454M [00:03<00:00, 145MB/s]
